# Dynamics of a Double Pendulum

In this module, we will study the dynamics of a double pendulum. We first derive the equations of motion from the Lagrangian. Next, we take a look at three different numerical integration schemes: Euler's method, Runge-Kutta, and a symplectic integrator. We will investigate how the initial conditions affect the behavior of the system, as well as some of the sources of error in these simulations. Finally, we will look at the properties of the system in phase space using phase plots and Poincare sections.

## Background

The double pendulum problem is a simple system which can produce surprisingly complex movements. It is a chaotic system, meaning it is unpredictable, and there is no closed form solution to the motion of the two masses. This is a frequent example problem in the study of dynamic systems, nonlinear controls, and mechanics.

In this notebook we will be treating the two rods as massless and ignoring the effects of friction. For a treatment of this problem without these assumptions, see: 

See the image below for the definition of constants.

![Image](figures/diagram.png)
#### Figure 1. Diagram of Double Pendulum System

## Equations of Motion

You are encouraged to follow this derivation ON PAPER on your own. The following is a method that will work in nearly any situation to find equations of motion, often working well where a force analysis becomes difficult. This is good practice for anyone studying mechanics. To derive the equations of motion, we will start with the Lagrangian. This is defined by:

$$\begin{equation}
L = T- U
\end{equation}$$

where $T$ is the total Kinetic Energy in the system, and $U$ is the total Potential Energy. Potential energy is calculated as:

$$\begin{equation}
U = m_1g(-y_1) + m_2g(-y_2)
\end{equation}$$

$$\begin{equation}
U = -m_1g(l_1cos(\theta_1) - m_2g(l_1cos\theta_1 + l_2cos\theta_2)
\end{equation}$$

Kinetic Energy is calculated as:

$$\begin{equation}
T = \frac{1}{2}mv_1^2+\frac{1}{2}mv_2^2
\end{equation}$$

$$\begin{equation}
T = \frac{1}{2}m_1(l_1\dot\theta_1)^2 + \frac{1}{2}m_2(l_1\dot\theta_1 + l_2\dot\theta_2)^2
\end{equation}$$

$$\begin{equation}
T = \frac{1}{2}(m_1+m_2)l_1^2\dot\theta_1^2+\frac{1}{2}m_2\left(2l_1l_2\dot\theta_1\dot\theta_2cos(\theta_2-\theta_1)+l_2^2\dot\theta_2^2\right)
\end{equation}$$


So the Lagrangian quantity becomes:
$$\begin{equation}
L = \frac{1}{2}(m_1+m_2)l_1^2\dot\theta_1^2+m_2l_1l_2\dot\theta_1\dot\theta_2cos(\theta_2-\theta_1)+(m_1+m_2)l_1gcos\theta_1 + m_2l_2gcos\theta_2
\end{equation}$$


The equations of motion are then found using the Lagrange Equation:

$$\begin{equation}
\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{x_i}}\right) - \frac{\partial L}{\partial x_i} = 0
\end{equation}$$


So the equation of motion for $\theta_1$ is calculated by the following steps:

$$\begin{equation}
\frac{\partial L}{\partial \dot{\theta_1}} = (m_1+m_2)l_1^2\dot\theta_1+m_2l_1l_2\dot\theta_2cos(\theta_2-\theta_1)
\end{equation}$$

$$\begin{equation}
\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\theta_1}}\right) = (m_1+m_2)l_1^2\ddot\theta_1+m_2l_1l_2\ddot\theta_2cos(\theta_2-\theta_1)-m_2l_1l_2\dot\theta_2^2sin(\theta_2-\theta_1)+m_2l_1l_2\dot\theta_1\dot\theta_2sin(\theta_2-\theta_1)
\end{equation}$$

$$\begin{equation}
\frac{\partial L}{\partial \theta_1} = m_2l_1l_2\dot\theta_1\dot\theta_2sin(\theta_2-\theta_1)+(m_1+m_2)gl_1sin\theta_1
\end{equation}$$

Some of these terms will cancel out, and we are left with a final equation of motion:

$$\begin{equation}
0 = (m_1+m_2)l_1^2\ddot\theta_1+m_2l_1l_2\ddot\theta_2cos(\theta_2-\theta_1)-m_2l_1l_2\dot\theta_2^2sin(\theta_2-\theta_1)+(m_1+m_2)gl_1sin\theta_1
\end{equation}$$

We will also need the equation of motion for $\theta_2$. This can be derived using the same process. It should work out to the following:

$$\begin{equation}
0 = m_2l_2^2\ddot\theta_2+m_2l_1l_2\ddot\theta_1cos(\theta_2-\theta_1) - m_2l_2l_1\dot\theta_1^2sin(\theta_2-\theta_1)+ l_2m_2gsin\theta_2
\end{equation}$$

## Problem setup

Notice that these equations of motion are implicit equations for $\ddot\theta_1$ and $\ddot\theta_2$. We have talked about implicit equations in this class, and we solve them by forming a system of equations. In this case, we will need to solve the equations of motion at each time step. We will characterize our system in state space using the angle and angular velocity of each pendulum. So our state becomes:

$$\begin{eqnarray}
\vec{x}(t)  = \begin{pmatrix} \theta_1\\ \theta_2\\ \dot\theta_1 \\ \dot\theta_2 \end{pmatrix}
\end{eqnarray}$$

The equations of motion can be rewritten as a system of equations:

$$ A\dot x=f(x)$$

where 

\begin{equation}
A = 
\left[\begin{array}{c}
1 &0 &0 &0 \\
0 &1 &0 &0 \\
0 &0 &(m_1+m_2)l_1^2 &m_2l_1l_2cos(x_2-x_1) \\
0 &0 &m_2l_1l_2cos(x_2-x_1) &m_2l_2^2
\end{array}\right]
\end{equation}

and

$$\begin{eqnarray}
f(\vec{x}(t))  = \begin{pmatrix} x_3\\ x_4\\ m_2l_1l_2x_4^2sin(x_2-x_1) -(m_1+m_2)gl_1sin(x_1)\\ m_2l_1l_2x_3^2sin(x_2-x_1)-l_2m_2gsinx_2 \end{pmatrix}
\end{eqnarray}$$

## Integration

#### Euler's Method

The first method we will use is Euler integration, covered in lesson 1 of the Numerical Methods course. This is the simplest form of numerical integration, but it can still be effective. Using our equations for $\dot x$ and $f(x)$:

$$x_i^{n+1} = x_i^n + h\dot{x}_i^n$$

where $h$ is the timestep.

#### Runge-Kutta Method (RK4)



## Solve!

Here we define functions for each of the integration methods, set our parameters and initial conditions, and then numerically integrate the motion of the pendulums. The plots below compare the different integration techniques.

In [4]:
# This cell loads the style of the notebook, which is taken from the 
# Numerical Methods in Python Course: http://openedx.seas.gwu.edu/courses/GW/MAE6286/2014_fall/about

from IPython.core.display import HTML
css_file = 'numericalmoocstyle.css'
HTML(open(css_file, "r").read())